In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
path = '../../../../titanic.csv'

df = pd.read_csv(path)

df.head()

,Unnamed: 0,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,0,3,male,22.0,1,0,7.2500,S,Third,NaN,Southampton,0
1,1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0
2,2,2,1,3,female,26.0,0,0,7.9250,S,Third,NaN,Southampton,1
3,3,3,1,1,female,35.0,1,0,53.1000,S,First,C,Southampton,0
4,4,4,0,3,male,35.0,0,0,8.0500,S,Third,NaN,Southampton,1


In [3]:
def prep_titanic(df):
    df.drop(['Unnamed: 0', 'passenger_id', 'embarked', 'pclass', 'deck'], axis=1, inplace=True)
    
    df.embark_town.fillna('Southampton', inplace=True)
    
    dummy1 = pd.get_dummies(df['sex'], dummy_na=False, drop_first=True)
    
    dummy2 = pd.get_dummies(df['embark_town'], dummy_na=False, drop_first=False)
    
    dummy3 = pd.get_dummies(df['class'], dummy_na=False, drop_first=False)
    
    df.drop(['sex', 'embark_town', 'class'], axis=1, inplace=True)
    
    df = pd.concat([df, dummy1, dummy2, dummy3], axis=1)
    
    return df

In [4]:
df = prep_titanic(df)

df.head()

,survived,age,sibsp,parch,fare,alone,male,Cherbourg,Queenstown,Southampton,First,Second,Third
0,0,22.0,1,0,7.2500,0,1,0,0,1,0,0,1
1,1,38.0,1,0,71.2833,0,0,1,0,0,1,0,0
2,1,26.0,0,0,7.9250,1,0,0,0,1,0,0,1
3,1,35.0,1,0,53.1000,0,0,0,0,1,1,0,0
4,0,35.0,0,0,8.0500,1,1,0,0,1,0,0,1


In [5]:
seed = 42

train, valid = train_test_split(df, test_size=0.2, random_state=seed,
                                stratify=df['survived'])

In [6]:
train.head()

,survived,age,sibsp,parch,fare,alone,male,Cherbourg,Queenstown,Southampton,First,Second,Third
692,1,NaN,0,0,56.4958,1,1,0,0,1,0,0,1
481,0,NaN,0,0,0.0000,1,1,0,0,1,0,1,0
527,0,NaN,0,0,221.7792,1,1,0,0,1,1,0,0
855,1,18.0,0,1,9.3500,0,0,0,0,1,0,0,1
801,1,31.0,1,1,26.2500,0,0,0,0,1,0,1,0


In [7]:
mms_age = MinMaxScaler()

In [8]:
train['age'] = mms_age.fit_transform(train[['age']])

train.head()

,survived,age,sibsp,parch,fare,alone,male,Cherbourg,Queenstown,Southampton,First,Second,Third
692,1,NaN,0,0,56.4958,1,1,0,0,1,0,0,1
481,0,NaN,0,0,0.0000,1,1,0,0,1,0,1,0
527,0,NaN,0,0,221.7792,1,1,0,0,1,1,0,0
855,1,0.220910,0,1,9.3500,0,0,0,0,1,0,0,1
801,1,0.384267,1,1,26.2500,0,0,0,0,1,0,1,0


In [9]:
mms_fare = MinMaxScaler()

In [10]:
train['fare'] = mms_fare.fit_transform(train[['fare']])

train.head()

,survived,age,sibsp,parch,fare,alone,male,Cherbourg,Queenstown,Southampton,First,Second,Third
692,1,NaN,0,0,0.110272,1,1,0,0,1,0,0,1
481,0,NaN,0,0,0.000000,1,1,0,0,1,0,1,0
527,0,NaN,0,0,0.432884,1,1,0,0,1,1,0,0
855,1,0.220910,0,1,0.018250,0,0,0,0,1,0,0,1
801,1,0.384267,1,1,0.051237,0,0,0,0,1,0,1,0


In [11]:
valid['age'] = mms_age.transform(valid[['age']])
valid['fare'] = mms_fare.transform(valid[['fare']])

In [12]:
valid.head()

,survived,age,sibsp,parch,fare,alone,male,Cherbourg,Queenstown,Southampton,First,Second,Third
565,0,0.296306,2,0,0.047138,0,1,0,0,1,0,0,1
160,0,0.547625,0,1,0.031425,0,1,0,0,1,0,0,1
553,1,0.271174,0,0,0.014102,1,1,1,0,0,0,0,1
860,0,0.509927,2,0,0.027538,0,1,0,0,1,0,0,1
241,1,NaN,1,0,0.030254,0,0,0,1,0,0,0,1


In [13]:
X_train = train.drop(columns=['survived'])
y_train = train['survived']

X_valid = valid.drop(columns=['survived'])
y_valid = valid['survived']

In [14]:
D_train = xgb.DMatrix(X_train, label = y_train)
D_valid = xgb.DMatrix(X_valid, label = y_valid)

In [15]:
params = {
    'verbosity': 1,
    'max_depth': 6,
    'objective': 'binary:logistic',
    'eta': 0.15,
    'random_state': seed
    }

steps = 100

In [16]:
model = xgb.train(params, D_train, steps,
                  evals=[(D_train, 'Train'), (D_valid, 'Valid')],
                  early_stopping_rounds=2)

[0]	Train-logloss:0.61424	Valid-logloss:0.63287
[1]	Train-logloss:0.55506	Valid-logloss:0.58662
[2]	Train-logloss:0.50858	Valid-logloss:0.55286
[3]	Train-logloss:0.47107	Valid-logloss:0.52662
[4]	Train-logloss:0.44140	Valid-logloss:0.50949
[5]	Train-logloss:0.41584	Valid-logloss:0.49613
[6]	Train-logloss:0.39517	Valid-logloss:0.48643
[7]	Train-logloss:0.37859	Valid-logloss:0.48107
[8]	Train-logloss:0.36348	Valid-logloss:0.47452
[9]	Train-logloss:0.35038	Valid-logloss:0.46783
[10]	Train-logloss:0.34084	Valid-logloss:0.46277
[11]	Train-logloss:0.33023	Valid-logloss:0.45874
[12]	Train-logloss:0.32226	Valid-logloss:0.45481
[13]	Train-logloss:0.31432	Valid-logloss:0.45477
[14]	Train-logloss:0.30751	Valid-logloss:0.45115
[15]	Train-logloss:0.30174	Valid-logloss:0.45036
[16]	Train-logloss:0.29553	Valid-logloss:0.45248
[17]	Train-logloss:0.28995	Valid-logloss:0.45647


In [17]:
model.attributes()

{'best_iteration': '15',
 'best_ntree_limit': '16',
 'best_score': '0.45035559833882244'}

In [18]:
model.get_fscore()

{'age': 164.0,
 'sibsp': 26.0,
 'parch': 14.0,
 'fare': 179.0,
 'alone': 1.0,
 'male': 18.0,
 'Cherbourg': 13.0,
 'Queenstown': 1.0,
 'Southampton': 12.0,
 'First': 9.0,
 'Second': 4.0,
 'Third': 22.0}

In [19]:
model.num_boosted_rounds()

18

In [20]:
y_hat = model.predict(D_valid)

In [21]:
y_hat = np.where(y_hat >= 0.5, 1, 0)

y_hat

array([0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0])

In [22]:
print(classification_report(y_valid, y_hat))

              precision    recall  f1-score   support

           0       0.80      0.88      0.84       110
           1       0.78      0.65      0.71        69

    accuracy                           0.79       179
   macro avg       0.79      0.77      0.77       179
weighted avg       0.79      0.79      0.79       179



In [23]:
classifier = XGBClassifier(n_estimators=100)

In [24]:
classifier.fit(X_train, y_train, early_stopping_rounds=2,
               eval_set=[(X_valid, y_valid)])

[0]	validation_0-logloss:0.58379
[1]	validation_0-logloss:0.52510
[2]	validation_0-logloss:0.49327
[3]	validation_0-logloss:0.47691
[4]	validation_0-logloss:0.46717
[5]	validation_0-logloss:0.46227
[6]	validation_0-logloss:0.46107
[7]	validation_0-logloss:0.45007
[8]	validation_0-logloss:0.45137


/opt/homebrew/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

# Light GBM on train dataset

In [2]:
path2 = '../../data/prepared/'

df = pd.read_csv(path2 + 'train_data.csv')

df.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938469,0.001733,0.008724,1.006838,0.009228,0.124035,0.008771,0.004709,...,NaN,NaN,NaN,0.002427,0.003706,0.003818,NaN,0.000569,0.000610,0.002674
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936665,0.005775,0.004923,1.000653,0.006151,0.126750,0.000798,0.002714,...,NaN,NaN,NaN,0.003954,0.003167,0.005032,NaN,0.009576,0.005492,0.009217
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28,0.954180,0.091505,0.021655,1.009672,0.006815,0.123977,0.007598,0.009423,...,NaN,NaN,NaN,0.003269,0.007329,0.000427,NaN,0.003429,0.006986,0.002603
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-06-13,0.960384,0.002455,0.013683,1.002700,0.001373,0.117169,0.000685,0.005531,...,NaN,NaN,NaN,0.006117,0.004516,0.003200,NaN,0.008419,0.006527,0.009600
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-07-16,0.947248,0.002483,0.015193,1.000727,0.007605,0.117325,0.004653,0.009312,...,NaN,NaN,NaN,0.003671,0.004946,0.008889,NaN,0.001670,0.008126,0.009827


In [3]:
df_labels = pd.read_csv(path2 + 'train_labels.csv')

df_labels.head()

,customer_ID,target
0,e039c846adcb1bfd543d89b8beba6a0f1a2ecdf835cd22...,0
1,8a96ff0aacb31ab4aab4cd15f13e024c04f986b8926789...,0
2,cfe6ef5945ef9c04937bb1826ec350ae7f2d173867f7fe...,0
3,9b677e823a94355cf65d2b97930dd59fce82cbf69e82b8...,1
4,d48853a03d0d463a4b058d129453daaf530160ccc4f0c2...,0


In [4]:
column_list = list(df.columns)

In [5]:
categorical_columns = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120',
                       'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

In [6]:
numerical_columns = [col for col in column_list if col not in categorical_columns]

In [7]:
numerical_columns.remove('customer_ID')

In [8]:
numerical_columns.remove('S_2')

In [9]:
numerical_columns[:5]

['P_2', 'D_39', 'B_1', 'B_2', 'R_1']

In [10]:
df.fillna(-2, inplace=True)

In [11]:
df.drop(columns=['S_2'], inplace=True)

In [12]:
df['customer_ID'].unique()

array(['0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a',
       '0005ef1480ffe4f0e5908eae07db8e05f2e705bca6a57db3b97d1da2eb6c7cab',
       '0005f2ab0cc352cd64eac9e60b981ed8c3289d20fa5fcef78973c468916fdfc9',
       ...,
       '8a1936a47736358988fdae1ce7c13d188d2adffd7a0d61d63d858ab7d1bef3d5',
       '8a196ee007f388857f00de28afe60ec8cbe8b06b45702ff7d23f6dd5fa9b785b',
       '8a19b2dd05e40a719a975d1c9e29ebab9d7f6603cfe17a418c22236d4891e270'],
      dtype=object)

In [13]:
df.head()

,customer_ID,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.938469,0.001733,0.008724,1.006838,0.009228,0.124035,0.008771,0.004709,-2.0,...,-2.0,-2.0,-2.0,0.002427,0.003706,0.003818,-2.0,0.000569,0.000610,0.002674
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.936665,0.005775,0.004923,1.000653,0.006151,0.126750,0.000798,0.002714,-2.0,...,-2.0,-2.0,-2.0,0.003954,0.003167,0.005032,-2.0,0.009576,0.005492,0.009217
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.954180,0.091505,0.021655,1.009672,0.006815,0.123977,0.007598,0.009423,-2.0,...,-2.0,-2.0,-2.0,0.003269,0.007329,0.000427,-2.0,0.003429,0.006986,0.002603
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.960384,0.002455,0.013683,1.002700,0.001373,0.117169,0.000685,0.005531,-2.0,...,-2.0,-2.0,-2.0,0.006117,0.004516,0.003200,-2.0,0.008419,0.006527,0.009600
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.947248,0.002483,0.015193,1.000727,0.007605,0.117325,0.004653,0.009312,-2.0,...,-2.0,-2.0,-2.0,0.003671,0.004946,0.008889,-2.0,0.001670,0.008126,0.009827


In [14]:
X = pd.DataFrame({'customer_ID': df['customer_ID'].unique()})

In [15]:
X.head()

,customer_ID
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...
1,0005ef1480ffe4f0e5908eae07db8e05f2e705bca6a57d...
2,0005f2ab0cc352cd64eac9e60b981ed8c3289d20fa5fce...
3,0007ee1ac8edb6be588c6c8a158d2268e3228c49cf5277...
4,0008ef32824d3067b4d8ab70e7c79fa04c808f4d99194f...


In [16]:
for col in categorical_columns:
    print(f'{col}')

B_30
B_38
D_114
D_116
D_117
D_120
D_126
D_63
D_64
D_66
D_68


In [17]:
for col in numerical_columns:
    mini = df.groupby('customer_ID')[col].min()
    maxi = df.groupby('customer_ID')[col].max()
    med = df.groupby('customer_ID')[col].median()
    std = df.groupby('customer_ID')[col].std()
    
    stats = pd.DataFrame({f'{col}_min': mini, f'{col}_max': maxi,
                       f'{col}_median': med, f'{col}_std': std})
    
    stats.reset_index(drop=True, inplace=True)
    X.reset_index(drop=True, inplace=True)
    
    X = pd.concat([X, stats], axis=1)

In [18]:
X.head()

,customer_ID,P_2_min,P_2_max,P_2_median,P_2_std,D_39_min,D_39_max,D_39_median,D_39_std,B_1_min,...,D_143_median,D_143_std,D_144_min,D_144_max,D_144_median,D_144_std,D_145_min,D_145_max,D_145_median,D_145_std
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.868580,0.960384,0.938469,0.024194,0.001082,0.091505,0.002483,0.024444,0.001930,...,0.005810,0.003564,0.000610,0.009616,0.005492,0.002598,0.000995,0.009827,0.006362,0.003294
1,0005ef1480ffe4f0e5908eae07db8e05f2e705bca6a57d...,0.068484,0.186780,0.127632,0.083648,0.031711,0.914384,0.473048,0.624144,1.318354,...,0.002345,0.000995,0.001506,0.009779,0.005643,0.005850,0.003469,0.008167,0.005818,0.003322
2,0005f2ab0cc352cd64eac9e60b981ed8c3289d20fa5fce...,0.837273,0.946487,0.894418,0.030962,0.004118,0.740627,0.413056,0.236028,0.015077,...,0.004801,0.003206,0.002932,0.009830,0.005286,0.002180,0.000746,0.008901,0.006505,0.002670
3,0007ee1ac8edb6be588c6c8a158d2268e3228c49cf5277...,0.365222,0.646557,0.417593,0.081273,0.000303,0.213325,0.006578,0.064191,0.085111,...,0.003010,0.003070,0.000296,0.007334,0.005844,0.002495,0.000517,0.008526,0.004997,0.002519
4,0008ef32824d3067b4d8ab70e7c79fa04c808f4d99194f...,0.752942,0.867515,0.843999,0.030110,0.000355,0.009881,0.007834,0.003731,0.000511,...,0.002951,0.555799,0.000128,0.008566,0.004376,0.002660,-2.000000,0.008376,0.004873,0.556017


In [44]:
X.shape

(21069, 710)

In [20]:
X = X.merge(df_labels, on='customer_ID')

In [21]:
train_X = X.drop(columns=['customer_ID', 'target'])
train_y = X['target']

In [22]:
D_train2 = xgb.DMatrix(train_X, label=train_y)

In [23]:
steps = 100
seed = 42

params = {
    'verbosity': 1,
    'max_depth': 6,
    'objective': 'binary:logistic',
    'eta': 0.15,
    'random_state': seed
    }

In [24]:
model = xgb.train(params, D_train2, steps,
                  evals=[(D_train2, 'Train')],
                  early_stopping_rounds=5)

[0]	Train-logloss:0.60127
[1]	Train-logloss:0.53140
[2]	Train-logloss:0.47667
[3]	Train-logloss:0.43264
[4]	Train-logloss:0.39651
[5]	Train-logloss:0.36713
[6]	Train-logloss:0.34209
[7]	Train-logloss:0.32094
[8]	Train-logloss:0.30296
[9]	Train-logloss:0.28705
[10]	Train-logloss:0.27384
[11]	Train-logloss:0.26201
[12]	Train-logloss:0.25185
[13]	Train-logloss:0.24287
[14]	Train-logloss:0.23544
[15]	Train-logloss:0.22873
[16]	Train-logloss:0.22214
[17]	Train-logloss:0.21619
[18]	Train-logloss:0.21139
[19]	Train-logloss:0.20659
[20]	Train-logloss:0.20211
[21]	Train-logloss:0.19791
[22]	Train-logloss:0.19456
[23]	Train-logloss:0.19096
[24]	Train-logloss:0.18765
[25]	Train-logloss:0.18451
[26]	Train-logloss:0.18170
[27]	Train-logloss:0.17900
[28]	Train-logloss:0.17563
[29]	Train-logloss:0.17331
[30]	Train-logloss:0.17101
[31]	Train-logloss:0.16855
[32]	Train-logloss:0.16614
[33]	Train-logloss:0.16422
[34]	Train-logloss:0.16140
[35]	Train-logloss:0.15896
[36]	Train-logloss:0.15656
[37]	Train-

In [25]:
y_pred = model.predict(D_train2)

In [26]:
y_pred

array([4.9524341e-04, 9.6756351e-01, 4.3878841e-04, ..., 1.5664291e-01,
       2.3160900e-03, 1.2846721e-03], dtype=float32)

In [27]:
y_pred.shape

(21069,)

In [28]:
y_train_final = pd.DataFrame(train_y)
y_pred_final = pd.DataFrame(y_pred, columns=['prediction'])

In [29]:
def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:

    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)

In [30]:
print(amex_metric(y_train_final, y_pred_final))

0.9962794290755908


# Validation dataset

In [59]:
valid_df = pd.read_csv(path2 + 'val_data.csv')
valid_labels = pd.read_csv(path2 + 'val_labels.csv')

In [60]:
valid_df.shape, valid_labels.shape

((1397562, 190), (214152, 2))

In [61]:
valid_df['customer_ID'].nunique()

115976

In [62]:
valid_df.fillna(-2, inplace=True)
valid_df.drop(columns=['S_2'], inplace=True)

In [63]:
X_valid = pd.DataFrame({'customer_ID': valid_df['customer_ID'].unique()})

In [64]:
valid_df.head()

,customer_ID,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0.929122,0.382790,0.025782,1.002647,0.005515,0.089799,0.001479,0.005830,-2.0,...,-2.0,-2.0,-2.0,0.003776,0.001808,0.008200,-2.0,0.001061,0.000027,0.002629
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0.918431,0.002224,0.006806,1.008186,0.003287,0.095966,0.007831,0.000861,-2.0,...,-2.0,-2.0,-2.0,0.004046,0.008288,0.001375,-2.0,0.009999,0.002669,0.006912
2,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0.922563,0.567403,0.033713,1.007497,0.005594,0.094655,0.003162,0.010466,-2.0,...,-2.0,-2.0,-2.0,0.007742,0.002693,0.009851,-2.0,0.000996,0.003254,0.004469
3,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0.919253,0.213193,0.031170,1.008534,0.008996,0.099089,0.009513,0.005360,-2.0,...,-2.0,-2.0,-2.0,0.002502,0.003399,0.004284,-2.0,0.002642,0.004325,0.006308
4,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0.901860,0.325467,0.109644,1.005098,0.008041,0.108526,0.000732,0.005484,-2.0,...,-2.0,-2.0,-2.0,0.000275,0.007992,0.006918,-2.0,0.009869,0.002979,0.005527


In [65]:
X_valid.head()

,customer_ID
0,00000fd6641609c6ece5454664794f0340ad84dddce9a2...
1,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...
2,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...
3,000084e5023181993c2e1b665ac88dbb1ce9ef621ec537...
4,0000f99513770170a1aba690daeeb8a96da4a39f11fc27...


In [66]:
len(numerical_columns)

177

In [67]:
for col in numerical_columns:
    mini = valid_df.groupby('customer_ID')[col].min()
    maxi = valid_df.groupby('customer_ID')[col].max()
    med = valid_df.groupby('customer_ID')[col].median()
    std = valid_df.groupby('customer_ID')[col].std()
    
    stats2 = pd.DataFrame({f'{col}_min': mini, f'{col}_max': maxi,
                       f'{col}_median': med, f'{col}_std': std})
    
    stats2.reset_index(drop=True, inplace=True)
    X_valid.reset_index(drop=True, inplace=True)
    
    X_valid = pd.concat([X_valid, stats2], axis=1)
    print('{} complete'.format(col))

P_2 complete
D_39 complete
B_1 complete
B_2 complete
R_1 complete
S_3 complete
D_41 complete
B_3 complete
D_42 complete
D_43 complete
D_44 complete
B_4 complete
D_45 complete
B_5 complete
R_2 complete
D_46 complete
D_47 complete
D_48 complete
D_49 complete
B_6 complete
B_7 complete
B_8 complete
D_50 complete
D_51 complete
B_9 complete
R_3 complete
D_52 complete
P_3 complete
B_10 complete
D_53 complete
S_5 complete
B_11 complete
S_6 complete
D_54 complete
R_4 complete
S_7 complete
B_12 complete
S_8 complete
D_55 complete
D_56 complete
B_13 complete
R_5 complete
D_58 complete
S_9 complete
B_14 complete
D_59 complete
D_60 complete
D_61 complete
B_15 complete
S_11 complete
D_62 complete
D_65 complete
B_16 complete
B_17 complete
B_18 complete
B_19 complete
B_20 complete
S_12 complete
R_6 complete
S_13 complete
B_21 complete
D_69 complete
B_22 complete
D_70 complete
D_71 complete
D_72 complete
S_15 complete
B_23 complete
D_73 complete
P_4 complete
D_74 complete
D_75 complete
D_76 complete
B_

In [68]:
X_valid.shape

(115976, 709)

In [69]:
X_valid.head()

,customer_ID,P_2_min,P_2_max,P_2_median,P_2_std,D_39_min,D_39_max,D_39_median,D_39_std,B_1_min,...,D_143_median,D_143_std,D_144_min,D_144_max,D_144_median,D_144_std,D_145_min,D_145_max,D_145_median,D_145_std
0,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0.861109,0.929122,0.904814,0.022119,0.002224,0.567403,0.211731,0.199150,0.006711,...,0.003979,0.003472,0.000027,0.009568,0.003169,0.002871,0.000796,0.009390,0.004469,0.002729
1,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0.797670,0.904482,0.884522,0.028911,0.000802,0.009704,0.004229,0.002758,0.001472,...,0.003332,0.002682,0.000129,0.009415,0.004747,0.003638,0.000443,0.006932,0.005110,0.002152
2,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0.805045,0.940382,0.879238,0.042325,0.000030,0.008680,0.004148,0.002882,0.000776,...,0.004662,0.002268,0.000633,0.009436,0.004070,0.002633,0.000083,0.009666,0.002884,0.003426
3,000084e5023181993c2e1b665ac88dbb1ce9ef621ec537...,0.765181,0.824061,0.784706,0.019167,0.007104,0.711983,0.066236,0.238607,0.007853,...,0.002796,0.003037,0.001140,0.009998,0.005668,0.002677,0.000672,0.009156,0.003767,0.002926
4,0000f99513770170a1aba690daeeb8a96da4a39f11fc27...,0.391495,0.424624,0.408131,0.009116,0.000269,0.647319,0.037644,0.209513,0.912386,...,1.002864,0.002029,0.000765,0.009371,0.004917,0.003242,0.727649,0.824853,0.732522,0.025978


In [70]:
valid_labels.head()

,customer_ID,target
0,fe0b6ed3f5ef7353e8980e5da72cf46f8cb80c3894dd76...,0
1,a35586d1ed04531a80199c76bbf32c8eab4c43f1f59c97...,0
2,e474d43060190cdc995616f0066f37b184f0628ab52d84...,0
3,c63e46f3dc819d25dcc9bba394452f7417894030b7ed41...,0
4,a54f7277592c93dabe51287296b23a79bfc1994f3d201d...,0


In [71]:
X_valid_final = X_valid.merge(valid_labels, on='customer_ID')

In [73]:
X_valid_final.head()

,customer_ID,P_2_min,P_2_max,P_2_median,P_2_std,D_39_min,D_39_max,D_39_median,D_39_std,B_1_min,...,D_143_std,D_144_min,D_144_max,D_144_median,D_144_std,D_145_min,D_145_max,D_145_median,D_145_std,target
0,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0.861109,0.929122,0.904814,0.022119,0.002224,0.567403,0.211731,0.199150,0.006711,...,0.003472,0.000027,0.009568,0.003169,0.002871,0.000796,0.009390,0.004469,0.002729,0
1,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0.797670,0.904482,0.884522,0.028911,0.000802,0.009704,0.004229,0.002758,0.001472,...,0.002682,0.000129,0.009415,0.004747,0.003638,0.000443,0.006932,0.005110,0.002152,0
2,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0.805045,0.940382,0.879238,0.042325,0.000030,0.008680,0.004148,0.002882,0.000776,...,0.002268,0.000633,0.009436,0.004070,0.002633,0.000083,0.009666,0.002884,0.003426,0
3,000084e5023181993c2e1b665ac88dbb1ce9ef621ec537...,0.765181,0.824061,0.784706,0.019167,0.007104,0.711983,0.066236,0.238607,0.007853,...,0.003037,0.001140,0.009998,0.005668,0.002677,0.000672,0.009156,0.003767,0.002926,0
4,0000f99513770170a1aba690daeeb8a96da4a39f11fc27...,0.391495,0.424624,0.408131,0.009116,0.000269,0.647319,0.037644,0.209513,0.912386,...,0.002029,0.000765,0.009371,0.004917,0.003242,0.727649,0.824853,0.732522,0.025978,1


In [75]:
X_valid_final.shape

(115976, 710)

In [76]:
valid_X = X_valid_final.drop(columns=['customer_ID', 'target'])
valid_y = X_valid_final['target']

In [77]:
D_valid2 = xgb.DMatrix(valid_X, label=valid_y)

In [78]:
y_valid_pred = model.predict(D_valid2)

In [79]:
y_valid_pred.shape

(115976,)

In [80]:
y_valid_final = pd.DataFrame(valid_y)
y_valid_pred_final = pd.DataFrame(y_valid_pred, columns=['prediction'])

In [81]:
print(amex_metric(y_valid_final, y_valid_pred_final))

0.7355707647710662
